This notebook is designed to showcase the power of [lvlspy](https://pypi.org/project/lvlspy/) as an API by using it replicate the results from [Gupta and Meyer](https://journals.aps.org/prc/abstract/10.1103/PhysRevC.64.025805). The paper focuses on calculating the equilibration rate between the isomeric state and the ground state of $^{26}\mathrm{Al}$. The methods built into [lvlspy](https://pypi.org/project/lvlspy/) are based on that publication.

To kick things off, we will quietly install any missing packages required for lvlspy and import all the packages required for this notebook. Most are built-in or installed with lvlspy.

In [ ]:
#importing system libraries
import sys,subprocess,importlib.util, io, requests

#checking and installing required packages
required  = {'lvlspy','matplotlib','tabulate'}
installed = {pkg for pkg in required if importlib.util.find_spec(pkg) is not None}
missing = required - installed

if missing:
    subprocess.check_call([sys.executable,'-m','pip','install','--quiet',*missing])

#import libraries and modules
import tabulate
import numpy as np
import lvlspy.spcoll as lc
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from lvlspy.io import xml
from lvlspy.calculate import isomer, evolve
from IPython.display import HTML, display, Math

Now that all the packages are installed, let us first download an XML file, found on [OSF](https://osf.io/dqzs9), and load into lvlspy

In [ ]:
new_coll = lc.SpColl()
xml.update_from_xml(new_coll,io.BytesIO(requests.get('https://osf.io/dqzs9/download').content))
#get the species into separate variable
al26 = new_coll.get()['al26']

Now that the species is loaded from the xml, let's calculate the isomerization rates then graph it

In [ ]:
T = np.logspace(8,10) #setting the temperature array

#initializing the rate arrays
l_21 = np.empty(len(T))
l_12 = np.empty(len(T))

for i,t in enumerate(T):
    l_12[i], l_21[i] = isomer.effective_rate(t,al26)

#Set the fontsize and graph
fontsize = 24

plt.figure(figsize = (8,8))
plt.rcParams['font.size'] = fontsize
plt.xscale('log')
plt.yscale('log')

plt.ylim([1.e-15,1.e+15])
plt.xlim([np.min(T)/1e+9,np.max(T)/1e+9])

plt.plot(T/1e+9,l_21,color = 'red',linewidth = 3,label = 'Isomerization Rate')

plt.ylabel(r'$\mathrm{\lambda_{21}^{eff} [s^{-1}]}$')
plt.xlabel(r'$\mathrm{T_{9}}$')

plt.yticks([1e-15,1e-10,1e-5,1e0,1e+5,1e+10,1e+15])

plt.axhline(y = 0.158,ls = '--',color = 'black',label = r'$0^{+}$ state $\beta^{-}$ Decay Rate')#beta decay rate

plt.legend(loc = 'lower right')
plt.show()

As we can see, the figure is a carbon copy of figure 1 from the [paper](https://journals.aps.org/prc/pdf/10.1103/PhysRevC.64.025805) without the removed level, which will be tackled now. Let's say you want see which levels are affecting the isomerization rate the most. lvlspy allows us to remove and levels in an easy manner.

In [ ]:
#re-using the previous graph with the original calculation

plt.figure(figsize = (8,8))
plt.rcParams['font.size'] = fontsize
plt.xscale('log')
plt.yscale('log')

plt.ylim([1.e-15,1.e+15])
plt.xlim([np.min(T)/1e+9,np.max(T)/1e+9])

plt.plot(T/1e+9,l_21,linewidth = 2,label = 'All Transitions')

plt.ylabel(r'$\mathrm{\lambda_{21}^{eff} [s^{-1}]}$')
plt.xlabel(r'$\mathrm{T_{9}}$')

plt.yticks([1e-15,1e-10,1e-5,1e0,1e+5,1e+10,1e+15])

plt.axhline(y = 1.e-1,ls = '--',color = 'black',label = r'$0^{+}$ state $\beta^{-}$ Decay Rate')

#removing transitions, graphing the result, then returning it before removing the next
levels = al26.get_levels()
for i in range(2,6):
    t_remove = al26.get_level_to_level_transition(levels[i],levels[1])
    #removing the transition
    al26.remove_transition(t_remove)
    lambda_21_eff = np.empty(len(T))
    lambda_12_eff = np.empty(len(T)) 
    for j,t in enumerate(T):
        lambda_12_eff[j],lambda_21_eff[j] = isomer.effective_rate(t,al26)
    #adding it back after the calculations
    al26.add_transition(t_remove)
    plt.plot(T/1e+9,lambda_21_eff,linewidth = 2,label = 'Transition '+str(i)+ ' to 1 disabled')

plt.legend(loc = 'center left',bbox_to_anchor=(1,0.5))
plt.show()

To clarify the graph, level 1 is the isomeric state and level 0 is the ground state. The transitions that most affect the isomerization rate are 2 $\rightarrow$ 1 and 3 $\rightarrow$ 1. This simple script can be used by theorists to guide experimentalists on which levels to get data on.

Now let's analyze the fugacity of the system. Fugacity here deviates away from the pressure definition it uses in thermodynamics, but the term was appropriated due to its description of a state's abundance likeliness to jump to another state with less fugacity. If two states have the same fugacity, they will not exchange abundances. Equilibrium is established when all states have the same fugacity of one.

In [ ]:
#setting up the initial conditions
y0 = np.zeros(len(levels))
y0[0] = 1.0 #setting the total abundance at the ground state

#resetting the temperature
T = 5e+9 #in K

#setting up the time array to be integrated over
time = np.logspace(-30,2,200)

y,fug = evolve.newton_raphson(al26,T,y0,time) #the method evolves the abundances and fugacities within the same method

#plotting the fugacities

plt.figure(figsize = (8,8))
plt.rcParams['font.size'] = fontsize

for i in range(fug.shape[0]):
    plt.plot(time,fug[i,:],label = 'Level '+str(i))

plt.xscale('log')
#plt.ylim([1e-25,1.25])
plt.xlim([1.e-20, 100])
plt.ylabel(r'Fugacity ($\phi$)')
plt.legend(loc = 'center left',bbox_to_anchor=(1,0.5))

plt.show()

The fugacities are expected to fall in between that of the isomer and the ground state. This would better be illustrated with an animation

In [ ]:
fig = plt.figure(figsize = (8,8))

#extract energies from the levels in the xml
E = np.empty(len(levels))
for i in range(len(E)):
    E[i] = levels[i].get_energy()

def updatefig(i):
    fig.clear()
    fig.suptitle(r'time (s): %8.2e; $T_{9}$ = %4.2f' %(time[i],T/1e+9))
    plt.scatter(fug[0,i],E[0],color = 'green') #ground state
    plt.scatter(fug[1,i],E[1],color = 'red') #isomer
    plt.scatter(fug[2:len(E)-1,i],E[2:len(E)-1],color = 'black')
    plt.axvline(x = fug[0,i],color = 'green')
    plt.axvline(x = fug[1,i],color = 'red')
    plt.xlabel(r'$\phi$')
    plt.ylabel('E (keV)')
    plt.xscale('log')
    plt.xlim([1e-50,10])
    plt.draw()

anim = animation.FuncAnimation(fig,updatefig,len(time))
display(HTML(anim.to_jshtml()))
plt.close() #closes the last frame

A few things that one can observe. Fristly, as expected, the fugacities between all the upper lying states falls between that of the ground state and that of the isomer. The second to note is that states with the same fugacity evolve together. The third is that equilibration occurs in less that 1e-9 seconds. 

The effective rate could be understood via a combinatorial interpretation. The expression $(f_{1}^{in})^{T}(I - F^{T})^{-1}f_{2}^{out}$ from $\lambda_{21}^{eff}$ is actually the effective branching ratio in terms of Graph Theory. The "cascade probability vectors" are defined as $\Gamma_{q}^{in} = (I - F)^{-1}f_{q}^{in}$. The term cascade refers to the de-excitation from higher states into state 'q'. The paper also defines the 'infinite'-arc generalization of $f_{q}^{out}$ with $\Gamma_{q}^{out} = (I - F^{T})^{-1}f_{q}^{out}$. They then go through the proof to rewrite $\lambda_{21}^{eff} = (\lambda_{2}^{out})^{T}\Gamma_{1}^{in}$. Here we sample over temperatures chosen in table 4 from the paper to compare the values of $W_{1}(T)$, $W_{2}(T)$, $G_{1}(T)$, and $G_{2}(T)$, as well as $\lambda_{21}^{eff}$. By comparing the values to those in the paper, there is good agreement in between the factors listed above for values of $T_{9}$ up to 1.0. The deviation with higher values of and about $T_{9}$ are due to the fact that the xml used contains only 22 levels while the paper's calculation is using 67.

In [ ]:
t9 = np.array([0.0100,0.0125,0.0150,0.0175,0.0200,0.0250,0.0300,0.0400,0.0500,0.0600,0.0700,0.0800,0.0900,0.1,
               0.125,0.15,0.175,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.25,1.5,1.75,2.0,2.5,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0])
n = len(t9)

table = np.empty((n,6))

for i in range(n):
    table[i,0],t_9 = t9[i], t9[i]*1e+9
    g_low_out,g_low_in,g_high_out,g_high_in = isomer.cascade_probabilities(t_9,al26)
    eqprob = al26.compute_equilibrium_probabilities(t_9)
    w_1,w_2,table[i,2],table[i,3],R_1k,R_2k,table[i,4],table[i,5] = ensemble_weights(levels,eqprob,g_low_in,g_high_in,l_low,l_high)
    l12,table[i,1] = lambda_effective(t_9,al26,l_low,l_high)

display(Math(r'\;\;\;\;\;\;T_{9} \;\;\;\;\;\;\;\lambda_{21}^{eff} (s^{-1})\;\;\;\;\;  W_{1}(T)\;\;\;\;\;\; W_{2}(T)\;\;\;\;\;\; G_{1}(T)\;\;\;\;\;\;\;G_{2}(T) '))
display(HTML(tabulate.tabulate(table,tablefmt='html',floatfmt=('.4f','.5e','.6f','.6f','.5f','.6f'))))